In [15]:
# ライブラリをインポート
import matplotlib.pyplot as plt
from google.cloud import speech
import io
import os
from pydub import AudioSegment
import ffmpeg

In [72]:
# 文字起こし関数(モノラルチャンネルver)
def transcription_wav(audio_file):
    # 音声ファイルの読み込み
    with io.open(audio_file, 'rb') as f:
        content = f.read()

    # APIパラメータの作成
    audio = speech.RecognitionAudio(content = content)
    config = speech.RecognitionConfig(
        # 都度エンコーディングする場合は、LINEAR16 しない場合は、ENCODING_UNSPECIFIED
        # hertzはwavファイルによっては変更する必要があるかもしれないです。自分の場合は32000に指定しろと言われました
        encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz = 8000,
        language_code = "ja-JP"
    )

    # APIの呼び出し
    client = speech.SpeechClient()
    response = client.recognize(config = config, audio = audio)

    # 結果の表示
    # for result in response.results:
    #     print(result.alternatives[0].transcript)
    for i, result in enumerate(response.results):
        print(result)
        print(i)
        # alternative = result.alternatives[0]
        # print('-'*20)
        # print('first alternative of result {}'.format(i))
        # print(u'Transcript:{}'.format(alternative.transcript))
        # print(u'Channel Tag:{}'.format(result.channel_tag))

In [62]:
# keyを指定
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../Transcription/key/credentials.json'
# 実際に実行したいファイルを指定し、文字起こし関数を実行
transcription_wav('sample1.wav')

--------------------
first alternative of result 0
Transcript:お世話なっております私 do you fine のお花と申しますお世話になっておりますそのための弊社側の設定変更ツールのま昨日契約と言うかこういうことできますかっていうの確認をしたいなと思いましてはい吉岡ハイツマニュアルには入ってるかいてはいたんですけれどもあの IP あるの設定なんですけどもえーとこれ自身はお願いしてもそれはにお願いをしてもできるし8こちら側てるかどうかだと言うか設定変更する側でこちらで確認できたを記入しても可能っていう認識で大丈夫ですかね
Channel Tag:0


In [18]:
import wave
import struct
from scipy import fromstring, int16
import numpy as np
import math

# wavファイルを59秒間隔で分割する関数
def cut_wav(audio_chang_wav, time, wav_cut_dir):
    wr = wave.open(audio_chang_wav, "r")

    # wav情報を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    total_time = 1.0 * fn / fr
    integer = math.floor(total_time)
    t = int(time)
    frames = int(ch * fr * t)
    # 小数点切り上げ（1分に満たない最後のシーンを出力するため）
    num_cut = int(math.ceil(integer / t))
    data = wr.readframes(wr.getnframes())
    wr.close()

    X = np.frombuffer(data, dtype=int16)

    for i in range(num_cut):
        outf = wav_cut_dir + str(i) + ".wav"
        start_cut = int(i * frames)
        end_cut = int(i * frames + frames)
        print(start_cut)
        print(end_cut)
        Y = X[start_cut:end_cut]
        outd = struct.pack("h" * len(Y), *Y)

        # 書き出し
        ww = wave.open(outf, "w")
        ww.setnchannels(ch)
        ww.setsampwidth(width)
        ww.setframerate(fr)
        ww.writeframes(outd)
        ww.close()
        print('終了しました')

In [19]:
# もしかすると、時間の長い音声ファイルは分割しなければ文字起こしできない可能性があるのでテスト
sound = AudioSegment.from_file('../Transcription/audio/ステレオaudio.wav', format='wav')
print(len(sound))
# msで記述されている（5~1000秒の切り出し）
sound1 = sound[5000:59000]
sound1.export('output.wav', format='wav')

584640


<_io.BufferedRandom name='output.wav'>

In [65]:
# APIに直接遅れるのが60秒未満なので、59秒で設定している
# 1分以上の場合は、uriでの扱いになる？
time = 59
audio_change_wav = 'sample1.wav'
wav_cut_dir = '../Transcription/test_file/'
# 音声ファイルをカットする関数
cut_wav(audio_change_wav,time,  wav_cut_dir)


0
472000
終了しました
472000
944000
終了しました
944000
1416000
終了しました
1416000
1888000
終了しました
1888000
2360000
終了しました
2360000
2832000
終了しました
2832000
3304000
終了しました
3304000
3776000
終了しました
3776000
4248000
終了しました
4248000
4720000
終了しました


In [38]:
import soundfile
# いただいたファイルの音声ファイルのbit数が8bitだったので変換時に16bitに変換しろとエラーが出た
# bit数を変換する関数
def bit_change(audio_file_path, subtype):
    data, fs = soundfile.read(audio_file_path)
    soundfile.write('../Transcription/audio/ステレオaudio_change.wav', data, fs, subtype=subtype)
    print('bit数の変換ファイルの出力が終了しました。')

In [39]:
audio_path = '../Transcription/audio/ステレオaudio.wav'
change_subtype = 'PCM_16'
# bit数を変換するための関数実行
bit_change(audio_path,change_subtype)

bit数の変換ファイルの出力が終了しました。


In [68]:
# ステレオチャンネルの音声の文字起こしをする関数
def transcription_wav_stereo(audio_file):
    client = speech.SpeechClient()

    with io.open(audio_file, 'rb') as f:
        content = f.read()

    audio = speech.RecognitionAudio(content=content)

    config = speech.RecognitionConfig(
        encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz = 8000,
        language_code = 'ja-JP',
        audio_channel_count = 2,
        enable_separate_recognition_per_channel = True,
    )

    response = client.recognize(config=config, audio=audio)

    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
        print('-'*20)
        print('first alternative of result {}'.format(i))
        print(u'Transcript:{}'.format(alternative.transcript))
        print(u'Channel Tag:{}'.format(result.channel_tag))

In [73]:
for i in range(0, 3):
    filename = '../Transcription/test_file/'+str(i)+'.wav'
    transcription_wav(filename)

alternatives {
  transcript: "\343\201\212\344\270\226\350\251\261\343\201\252\343\201\243\343\201\246\343\201\212\343\202\212\343\201\276\343\201\231\347\247\201 do you fine \343\201\256\343\201\212\350\212\261\343\201\250\347\224\263\343\201\227\343\201\276\343\201\231\343\201\212\344\270\226\350\251\261\343\201\253\343\201\252\343\201\243\343\201\246\343\201\212\343\202\212\343\201\276\343\201\231\343\201\235\343\201\256\343\201\237\343\202\201\343\201\256\345\274\212\347\244\276\345\201\264\343\201\256\350\250\255\345\256\232\345\244\211\346\233\264\343\203\204\343\203\274\343\203\253\343\201\256\343\201\276\346\230\250\346\227\245\345\245\221\347\264\204\343\201\250\350\250\200\343\201\206\343\201\213\343\201\223\343\201\206\343\201\204\343\201\206\343\201\223\343\201\250\343\201\247\343\201\215\343\201\276\343\201\231\343\201\213\343\201\243\343\201\246\343\201\204\343\201\206\343\201\256\347\242\272\350\252\215\343\202\222\343\201\227\343\201\237\343\201\204\343\201\252\343\201\

In [76]:
from pydub import AudioSegment
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write

# ステレオチャンネルの音声をleft, rightで分割して出力する関数
def stereo_channel_split(stereo_audio):
    sound = AudioSegment.from_file(stereo_audio)
    # チャンネル数
    channel_count = sound.channels
    # frame_rate = speech to text でいうところのhertz
    frames_per_second = sound.frame_rate
    # ファイルの音声の長さ（秒）
    duration = sound.duration_seconds
    # 音声ファイルをnumpyで変換
    sound_array = np.array(sound.get_array_of_samples())
    # 右左に分割
    left_sound = sound_array[0:len(sound_array):2]
    right_sound = sound_array[1:len(sound_array):2]
    # 音声データの書き出し
    write('left_audio.wav', frames_per_second, left_sound)
    write('right_audio.wav', frames_per_second, right_sound)
    print('音声の分割が終了しました')

# 試しに実行
audio_path = '../Transcription/audio/ステレオaudio_change.wav'
stereo_channel_split(audio_path)